In [1]:
#importing the required modules
import os
import time
import pandas as pd

In [5]:
#get a merged list for all the informations about alignments
start = time.time()
features = []
with open (r'Z:\balint\numt\Michael\data/significant_alignments.fasta') as infile:
    content = infile.readlines()
    for index, line in enumerate(content):
        if 'score' in line:
            score = int(line.rsplit()[1].split('=')[1])
            eg2_value = float(line.rsplit()[2].split('=')[1])
            e_value = float(line.rsplit()[3].split('=')[1])
            #genomic informations
            genomic = content[index + 1]
            genomic_id = genomic.rsplit()[1]
            genomic_start = int(genomic.rsplit()[2])
            genomic_length = int(genomic.rsplit()[3])
            genomic_strand = genomic.rsplit()[4]
            genomic_size = int(genomic.rsplit()[5])
            genomic_sequence = genomic.rsplit()[6]
            #mitochondrial informations
            mitochondrial = content[index + 2]
            mitochondrial_start = int(mitochondrial.rsplit()[2])
            mitochondrial_length = int(mitochondrial.rsplit()[3])
            mitochondrial_strand = mitochondrial.rsplit()[4]
            mitochondrial_size = int(mitochondrial.rsplit()[5])
            mitochondrial_sequence = mitochondrial.rsplit()[6]
            features.append([score, eg2_value, e_value, genomic_id, genomic_start,
                            mitochondrial_start, genomic_length, mitochondrial_length, genomic_strand,
                            mitochondrial_strand, genomic_size, mitochondrial_size, genomic_sequence,
                             mitochondrial_sequence])
end = time.time()
runtime = (end - start) / 60#minutes
runtime

0.00926966667175293

In [7]:
#create dataframe and name columns
alignment_df = pd.DataFrame(features)
alignment_df.columns = ['score', 'eg2_value', 'e_value', 'g_id', 'g_start', 'mt_start',
                        'g_length', 'mt_length', 'g_strand', 'mt_strand', 'g_size', 'mt_size',
                        'g_sequence', 'mt_sequence']

In [8]:
alignment_df.head()

,score,eg2_value,e_value,g_id,g_start,mt_start,g_length,mt_length,g_strand,mt_strand,g_size,mt_size,g_sequence,mt_sequence
0,17245,0.0,0.0,MT,0,0,17245,17245,+,+,17245,34490,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...
1,17245,0.0,0.0,MT,0,17245,17245,17245,+,+,17245,34490,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...
2,3398,0.0,0.0,1,114731432,6976,4680,4681,+,+,194850757,34490,taGTAAAATTATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,TAGTAAAACCATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...
3,3398,0.0,0.0,1,114731432,24221,4680,4681,+,+,194850757,34490,taGTAAAATTATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,TAGTAAAACCATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...
4,3120,0.0,0.0,GL019308,67160,883,4892,4945,+,+,88228,34490,CAAGGCCATGAAGCATGCACACACTGCCCATCGCTCTCCTCAAACA...,CAAGGCCATGAAGCACGCACACACCGCCCGTCACCCTCCTCAAGTG...


In [10]:
alignment_df.to_csv(r'Z:\balint\numt\Michael\data/lastal_array.csv', index = False)

In [68]:
complement_dictionary = {'A':'T',
             'C':'G',
             'T':'A',
             'G':'C'}

In [13]:
#this is how we can get back 16223
(len(mt_full_seq) - alignment_df[mask1]['mt_start']) - alignment_df[mask1]['mt_length']

289    16223
dtype: int64

In [62]:
#parse alignment file for upstream and downstream flankings
time_start = time.time()
fasta_directory = os.path.join(r'D:\balint\munts\chromosomes_and_contigs/')
upstream_flankings = []
downstream_flankings = []
for index, row in alignment_df.iterrows():
    filename = '%s.fasta' % row['g_id']
    start = int(row['g_start'])
    length = int(row['g_length'])
    end = start + length
    flank_start = start - 200
    flank_end = end + 200
    with open (fasta_directory + filename) as infile:
        sequence = infile.readlines()[1:][0]
        if flank_start < 0:
            up_flank = sequence[ : start]
        else:
            up_flank = sequence[flank_start : start]
        if flank_end > len(sequence):
            down_flank = sequence[end : ]
        else:
            down_flank = sequence[end : flank_end]
        if up_flank == '':
            up_flank = float('nan')
        if down_flank == '':
            down_flank = float('nan')
        upstream_flankings.append(up_flank)
        downstream_flankings.append(down_flank)
        if index % 30 == 0:
            print('done for %s numts' % index)
time_end = time.time()
runtime = (time_end - time_start) / 60#minutes
runtime

done for 0 numts
done for 30 numts
done for 60 numts
done for 90 numts
done for 120 numts
done for 150 numts
done for 180 numts
done for 210 numts
done for 240 numts
done for 270 numts


2.1954854925473533

In [65]:
#adding flanking columns to the dataframe
alignment_df['down_flank'] = downstream_flankings
alignment_df['up_flank'] = upstream_flankings

In [66]:
alignment_df.head()

,score,eg2_value,e_value,g_id,mt_id,g_start,mt_start,g_length,mt_length,g_strand,mt_strand,g_size,mt_size,g_sequence,mt_sequence,down_flank,up_flank
0,17245,0,0,MT,MT,0,0,17245,17245,+,+,17245,34490,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,NaN,NaN
1,17245,0,0,MT,MT,0,17245,17245,17245,+,+,17245,34490,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,NaN,NaN
2,3398,0,0,1,MT,114731432,6976,4680,4681,+,+,194850757,34490,taGTAAAATTATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,TAGTAAAACCATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,TATATATATTTATATATATATGTATATATGTACATATATATAACTA...,ACAAAAGACTCTGAATAGCCAAAGCGATCCTGAACAAGAAAAATCA...
3,3398,0,0,1,MT,114731432,24221,4680,4681,+,+,194850757,34490,taGTAAAATTATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,TAGTAAAACCATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,TATATATATTTATATATATATGTATATATGTACATATATATAACTA...,ACAAAAGACTCTGAATAGCCAAAGCGATCCTGAACAAGAAAAATCA...
4,3120,0,0,GL019308,MT,67160,883,4892,4945,+,+,88228,34490,CAAGGCCATGAAGCATGCACACACTGCCCATCGCTCTCCTCAAACA...,CAAGGCCATGAAGCACGCACACACCGCCCGTCACCCTCCTCAAGTG...,ATGAAGGGGCAAAGAGTAATTTTACAGTCGAGAGGGCTAGCAGTCA...,GATAAACCCCGATAAACTTTACCACTCTTTGCCAACTCAGCCTCTA...


In [67]:
alignment_df.to_csv(r'Z:\balint\numt\Michael\data/alignments.csv', index = False)

In [64]:
trial_dir = os.path.join(r'Z:\balint\numt\Michael\data\trial_sequences/')
with open (trial_dir + 'trial_genome.fasta', 'w') as g, open (trial_dir + 'tiral_mito.fasta', 'w') as m:
    g.write('>1\n')
    g.write('AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGTGTGTGTGTGCCCCCCCCCCCCGGGGGTTTTTTTCCCCCCACACACACACACACACACACA\n')
    g.write('>2\n')
    g.write('TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTCACACACACACGGGGGGGGGGGGCCCCCAAAAAAAGGGGGGTGTGTGTGTGTGTGTGTGTGT\n')
    m.write('>M\n')
    m.write('AAAAGTGTGTGTGTGCCCCCCCCCCCCGGGGGTTTTTTTCCCCCCACAC')

In [63]:
len('TGTGTGTGTGTGTGTGTGTGT')

21